# Orality vs. Literacy in Dialogue

In this notebook, we will explore the relationship between patterns of oral language and patterns of written language, creating a classifier with contemporary sources and applying it to corpora from other time periods as well. 

In [ ]:
import json
import nltk
import sklearn
from scipy import sparse
from collections import Counter
import operator
import spacy
import gensim

## Data Cleaning

In [ ]:
##Data Parsing COCA here
##NER for name detection and stopword prevention (src: Mauro Di Pietro)
## predict wit NER
txt = dtf["text"].iloc[0]
entities = ner(txt).ents
## tag text
tagged_txt = txt
for tag in entities:
    tagged_txt = re.sub(tag.text, "_".join(tag.text.split()), tagged_txt) 
## show result
print(tagged_txt)

## Making Features

*Word Frequency*

In [ ]:
## src: Mauro Di Pietro
## need lst_words
## count
lst_grams = [len(word.split(" ")) for word in lst_words]
vectorizer = feature_extraction.text.CountVectorizer(
                 vocabulary=lst_words, 
                 ngram_range=(min(lst_grams),max(lst_grams)))
dtf_X = pd.DataFrame(vectorizer.fit_transform(dtf["text_clean"]).todense(), columns=lst_words)
## add the new features as columns
dtf = pd.concat([dtf, dtf_X.set_index(dtf.index)], axis=1)
dtf.head()

*Topic Modeling*

In [1]:
##To use this with a train/test CV split, we have to do a bit more work. see below: 
## https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28

In [ ]:
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
##This is not a cleaned version of the pipeline, cut from Classification.ipynb
def pipeline(trainX, devX, trainY, devY, feature_functions):

    """ This function evaluates a list of feature functions on the training/dev data arguments """
    
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data.
    feature_vocab=create_vocab(trainX_feat, top_n=100000)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    clf = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000)
    clf.fit(trainX_ids, trainY)
    print("Accuracy: %.3f" % clf.score(devX_ids, devY))
    
    return clf, feature_vocab

In [ ]:
##This is not a cleaned version of the pipeline, cut from Classification.ipynb
features=[your_awesome_feature]
pipeline(trainX, devX, trainY, devY, features)